# Práctica: Eliminación Contextual de Stopwords para Análisis de Reseñas de Productos

**Contexto:** Eres un NLP Engineer en **ReviewBoost**, una startup que analiza reseñas de Amazon para identificar problemas críticos en productos.

## Fase 1: Análisis Inicial

**Objetivo:** Entender cómo las stopwords por defecto afectan el significado en reseñas.

In [ ]:

import pandas as pd
import spacy
from collections import Counter

# Cargar dataset
df = pd.read_csv("comensarios_clintes.csv")
nlp = spacy.load("es_core_news_sm")

# Procesar texto y eliminar stopwords por defecto
example = "No recomiendo este producto. Aunque es barato, se rompió en dos días."
doc = nlp(example)
tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
print("Tokens tras stopwords por defecto:", tokens)

# Mostrar stopwords eliminadas
removed = [token.text for token in doc if token.is_stop]
print("Stopwords eliminadas:", removed)


**Verificación:** Generar una tabla con 5 ejemplos donde la eliminación de stopwords alteró el significado.

## Fase 2: Personalización de la Lista

**Objetivo:** Crear una lista de stopwords adaptada a reseñas de productos.

In [ ]:

# Construir lista personalizada
stopwords_es = nlp.Defaults.stop_words.copy()

# Preservar negaciones y conectores
for term in ["no","nunca","tampoco","pero","aunque","sin embargo"]:
    stopwords_es.discard(term)

# Añadir términos genéricos y no informativos
for term in ["producto","cliente","día","hola","gracias","pd"]:
    stopwords_es.add(term)

print("Tamaño lista stopwords personalizada:", len(stopwords_es))


## Fase 3: Implementación y Pruebas

**Objetivo:** Validar que la lista personalizada preserva el contexto crítico.

In [ ]:

# Función de procesamiento con stopwords personalizadas
def procesar_texto(texto, stopwords):
    doc = nlp(texto)
    return [token.text.lower() for token in doc if token.text.lower() not in stopwords and not token.is_punct]

# Casos de prueba
casos = {
    "No funciona bien, pero el diseño es bonito.": ["no","funciona","bien","pero","diseño","bonito"],
    "Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale.": ["nunca","compré","malo","aunque","precio","bajo","no","vale"]
}

for texto, esperado in casos.items():
    resultado = procesar_texto(texto, stopwords_es)
    print(f"Input: {texto}\nOutput: {resultado}\nEsperado: {esperado}\n")


**Verificación:** Debe preservarse 100% de términos clave en 20 casos de prueba.

## Fase 4: Evaluación de Impacto

**Objetivo:** Medir cómo afecta la personalización al análisis de sentimiento.

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Supongamos que df tiene columnas 'texto' y 'etiqueta'
# Crear columnas procesadas
df['texto_default'] = df['texto'].apply(lambda x: " ".join([t.text.lower() for t in nlp(x) if not t.is_stop and not t.is_punct]))
df['texto_custom'] = df['texto'].apply(lambda x: " ".join(procesar_texto(x, stopwords_es)))

# Vectorización
vec = CountVectorizer(max_features=1000)
X_def = vec.fit_transform(df['texto_default'])
X_cus = vec.fit_transform(df['texto_custom'])
y = df['etiqueta']

# Split
X_train_def, X_test_def, y_train, y_test = train_test_split(X_def, y, test_size=0.2, random_state=42)
X_train_cus, X_test_cus, _, _ = train_test_split(X_cus, y, test_size=0.2, random_state=42)

# Modelos
model = LogisticRegression()
model.fit(X_train_def, y_train)
acc_def = accuracy_score(y_test, model.predict(X_test_def))

model.fit(X_train_cus, y_train)
acc_cus = accuracy_score(y_test, model.predict(X_test_cus))

print("Accuracy sin personalización:", acc_def)
print("Accuracy con personalización:", acc_cus)


## Entrega Final

1. Script `stopwords_custom.py` con funciones de carga, procesamiento y evaluación.
2. Archivo `.txt` con lista final de stopwords.
3. Reporte con resultados de evaluación.